In [2]:
import os
import pandas as pd

# Path to your CSV file
csv_file = 'data-v5/test/_annotations.csv'

# Path to the directory containing the images to exclude
exclude_dir = 'data-v5/test/their_test'

# Path to save the excluded rows
excluded_csv = 'data-v5/test/their_test/_annotations.csv'

# Get a list of image filenames (without path) in the exclude directory
exclude_images = {os.path.basename(file) for file in os.listdir(exclude_dir)}

# Load the CSV file into a DataFrame
df = pd.read_csv(csv_file)

# Split the DataFrame into rows to keep and rows to remove
df_to_remove = df[df['filename'].isin(exclude_images)]
filtered_df = df[~df['filename'].isin(exclude_images)]

# Save the filtered DataFrame back to the CSV file
filtered_df.to_csv(csv_file, index=False)

# Save the removed rows into a new CSV file
df_to_remove.to_csv(excluded_csv, index=False)

print(f"Rows associated with images in {exclude_dir} have been removed.")

Rows associated with images in data-v5/test/their_test have been removed.
